In [23]:
# library here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [24]:
# read data and concat them
df_ds = pd.read_csv("/content/LinedIn_Data_scientists.csv")
df_an = pd.read_csv("/content/LinedIn_Data_Analysts.csv")
df_de = pd.read_csv("/content/LinedIn_Data_Engineer.csv")

df = pd.concat([df_ds, df_an, df_de], ignore_index=True, axis=0)

In [25]:
# check the info of data without preprocessing
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7123 entries, 0 to 7122
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        7120 non-null   object
 1   company_name     7120 non-null   object
 2   time_posted      6627 non-null   object
 3   num_applicants   2572 non-null   object
 4   salary           2032 non-null   object
 5   location         7120 non-null   object
 6   description      7120 non-null   object
 7   Seniority level  7120 non-null   object
 8   Employment type  7120 non-null   object
 9   Job function     7120 non-null   object
 10  Industries       7120 non-null   object
dtypes: object(11)
memory usage: 612.3+ KB


In [26]:
# check the # of duplicated and delete them
print(df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)

6424


In [27]:
# info of dataset after removing duplicates
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        698 non-null    object
 1   company_name     698 non-null    object
 2   time_posted      643 non-null    object
 3   num_applicants   266 non-null    object
 4   salary           207 non-null    object
 5   location         698 non-null    object
 6   description      698 non-null    object
 7   Seniority level  698 non-null    object
 8   Employment type  698 non-null    object
 9   Job function     698 non-null    object
 10  Industries       698 non-null    object
dtypes: object(11)
memory usage: 60.2+ KB


In [28]:
# create new column "Number of duplicates"
# after fill all null values
# df["Num of duplicates"] = df.groupby(df.columns.to_list()).transform('size')-1
# df

## location

In [29]:
# split "location" column into "city" and "state" columns
df[["city", "state"]] = df["location"].str.split(",", expand=True)
df.loc[df["city"]=="Remote", "state"] = "Remote"
df.loc[df["city"]=="United States", "state"] = "United States"
df.drop(columns=["location"], inplace=True)
df.head()

# df["city"].unique()

,job_title,company_name,time_posted,num_applicants,salary,description,Seniority level,Employment type,Job function,Industries,city,state
0,Data Scientists,Further,1 month ago,155 applicants,NaN,If you love data and are looking for unlimited...,Entry level,Full-time,Engineering and Information Technology,Business Consulting and Services,Cleveland,OH
1,Data Scientist(Junior),SynergisticIT,5 months ago,31 applicants,NaN,"At SynergisticIT, we aim to bring aboard IT pr...",Entry level,Part-time,Engineering and Information Technology,IT Services and IT Consulting,Ohio,United States
2,Junior Data Scientist - Remote,SynergisticIT,5 months ago,NaN,NaN,Since 2010 Synergisticit has helped Jobseekers...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH
3,Data Scientist I,ConstructConnect,1 week ago,NaN,NaN,This position sits within our Product Developm...,Entry level,Full-time,Engineering and Information Technology,Software Development,Cincinnati,OH
4,Data Scientist(Remote),SynergisticIT,5 months ago,NaN,NaN,Synergistic IT is a full-service staffing and ...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH


In [30]:
df.dropna(subset=["job_title"], inplace=True)

## time posted

In [31]:
# function of unifying the units
def adjust_time_posted(x):
  if pd.notna(x):
    time_posted = x.split()
    num = int(time_posted[0])
    unit = time_posted[1]
    if "year" in unit:
      return num*365
    elif "month" in unit:
      return num*30
    elif "week" in unit:
      return num*7
    elif "day" in unit:
      return num
    else:
      return np.nan

In [32]:
# apply the adjust_time_posted function to "time posted" column
df["adjusted time_posted"] = df["time_posted"].apply(adjust_time_posted)

# remove "time posted" column
df.drop("time_posted", axis=1, inplace=True)
# If the values in adjusted time posted are null, fill them with the mean
df["adjusted time_posted"] = df["adjusted time_posted"].fillna(df["adjusted time_posted"].mean())

## num applicants

In [33]:
# conver data type from obj to numeric
df["num_applicants"] = df["num_applicants"].str.replace("applicants", "").astype(float)
df.head()

,job_title,company_name,num_applicants,salary,description,Seniority level,Employment type,Job function,Industries,city,state,adjusted time_posted
0,Data Scientists,Further,155.0,NaN,If you love data and are looking for unlimited...,Entry level,Full-time,Engineering and Information Technology,Business Consulting and Services,Cleveland,OH,30.0
1,Data Scientist(Junior),SynergisticIT,31.0,NaN,"At SynergisticIT, we aim to bring aboard IT pr...",Entry level,Part-time,Engineering and Information Technology,IT Services and IT Consulting,Ohio,United States,150.0
2,Junior Data Scientist - Remote,SynergisticIT,NaN,NaN,Since 2010 Synergisticit has helped Jobseekers...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH,150.0
3,Data Scientist I,ConstructConnect,NaN,NaN,This position sits within our Product Developm...,Entry level,Full-time,Engineering and Information Technology,Software Development,Cincinnati,OH,7.0
4,Data Scientist(Remote),SynergisticIT,NaN,NaN,Synergistic IT is a full-service staffing and ...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH,150.0


## salary column

In [34]:
# function to unify the unit
def adjust_salary(x):
    if pd.notna(x): # Not null
        if x < 1000:
            return x * 1767
        else:
            return x
    return x

In [35]:
df["salary"].unique()
df["salary"] = df["salary"].str.replace("/yr", "")
df["salary"] = df["salary"].str.replace("/hr", "")
df["salary"] = df["salary"].str.replace("₹", "")
df["salary"] = df["salary"].str.replace("$", "")
df["salary"] = df["salary"].str.replace(",", "")
# df["salary"].unique()
df[["min salary", "max salary"]] = df["salary"].str.split("-", expand=True)
df["min salary"] = pd.to_numeric(df["min salary"])
df["max salary"] = pd.to_numeric(df["max salary"])
df["min salary"] = df["min salary"].apply(adjust_salary)
df["max salary"] = df["max salary"].apply(adjust_salary)
df.drop(columns=["salary"], axis=1, inplace=True)
df

,job_title,company_name,num_applicants,description,Seniority level,Employment type,Job function,Industries,city,state,adjusted time_posted,min salary,max salary
0,Data Scientists,Further,155.0,If you love data and are looking for unlimited...,Entry level,Full-time,Engineering and Information Technology,Business Consulting and Services,Cleveland,OH,30.000000,NaN,NaN
1,Data Scientist(Junior),SynergisticIT,31.0,"At SynergisticIT, we aim to bring aboard IT pr...",Entry level,Part-time,Engineering and Information Technology,IT Services and IT Consulting,Ohio,United States,150.000000,NaN,NaN
2,Junior Data Scientist - Remote,SynergisticIT,NaN,Since 2010 Synergisticit has helped Jobseekers...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH,150.000000,NaN,NaN
3,Data Scientist I,ConstructConnect,NaN,This position sits within our Product Developm...,Entry level,Full-time,Engineering and Information Technology,Software Development,Cincinnati,OH,7.000000,NaN,NaN
4,Data Scientist(Remote),SynergisticIT,NaN,Synergistic IT is a full-service staffing and ...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH,150.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,Power BI Dashboard Analyst Developer,Tata Consultancy Services,NaN,Technical/Functional Skills Power BI Dashboard...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",IT Services and IT Consulting,Columbus,OH,7.000000,95000.0,120000.0
695,Software Engineer Opportunities,Central Insurance,NaN,"Location: Hybrid Work Model – Van Wert, Ohio o...",Entry level,Full-time,Engineering and Information Technology,Insurance,Van Wert,OH,7.000000,NaN,NaN
696,Software Quality Assurance (QA) Test Engineer,Medpace,NaN,Voted a Top Cincinnati Workplace for 4 years r...,Not Applicable,Full-time,Quality Assurance,Pharmaceutical Manufacturing,Cincinnati,OH,32.267496,NaN,NaN
697,Software Engineer II,Honeywell,NaN,"Join a team recognized for leadership, innovat...",Entry level,Full-time,Engineering and Information Technology,"Appliances, Electrical, and Electronics Manufa...",Mason,OH,14.000000,NaN,NaN


In [36]:
# check the # of null
df.isna().sum()

,0
job_title,0
company_name,0
num_applicants,432
description,0
Seniority level,0
Employment type,0
Job function,0
Industries,0
city,0
state,15


In [37]:
# fill the null values with the mean
df["num_applicants"] = df["num_applicants"].fillna(df["num_applicants"].mean())

In [38]:
# fill the null values with "Other"
df["city"] = df["city"].fillna("Other")
df["state"] = df["state"].fillna("Other")

In [39]:
# fill the null values with the mean
df["adjusted time_posted"] = df["adjusted time_posted"].fillna(df["adjusted time_posted"].mean())

In [40]:
# fill the null values depending on the city and Industries
# calculate the min & max salry depending on the city and Industries
avg_min_salary_per_city_industry = df.groupby(["city", "Industries"])["min salary"].mean()
avg_max_salary_per_city_industry = df.groupby(["city", "Industries"])["max salary"].mean()

# fill the null values
df["min salary"] = df.apply(
  # get the avg min salary based on "city" and "Industries"
  lambda x: avg_min_salary_per_city_industry.get((x["city"], x["Industries"]), x["min salary"])
  # if "min salary" is null, replace it with the avg min salary
  if pd.isna(x["min salary"]) else x["min salary"], axis=1
)

df["max salary"] = df.apply(
  # get the avg max salary based on "city" and "Industries"
  lambda x: avg_max_salary_per_city_industry.get((x["city"], x["Industries"]), x["max salary"])
  # if "max salary" is null, replace it with the avg max salary
  if pd.isna(x["max salary"]) else x["max salary"], axis=1
)

In [41]:
# if the null values are not replaced, fill them with the median
df["min salary"] = df["min salary"].fillna(df["min salary"].median())
df["max salary"] = df["max salary"].fillna(df["max salary"].median())

In [42]:
df

,job_title,company_name,num_applicants,description,Seniority level,Employment type,Job function,Industries,city,state,adjusted time_posted,min salary,max salary
0,Data Scientists,Further,155.000000,If you love data and are looking for unlimited...,Entry level,Full-time,Engineering and Information Technology,Business Consulting and Services,Cleveland,OH,30.000000,92500.000000,100000.000000
1,Data Scientist(Junior),SynergisticIT,31.000000,"At SynergisticIT, we aim to bring aboard IT pr...",Entry level,Part-time,Engineering and Information Technology,IT Services and IT Consulting,Ohio,United States,150.000000,103327.000000,182983.000000
2,Junior Data Scientist - Remote,SynergisticIT,85.699248,Since 2010 Synergisticit has helped Jobseekers...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH,150.000000,100977.222222,126081.444444
3,Data Scientist I,ConstructConnect,85.699248,This position sits within our Product Developm...,Entry level,Full-time,Engineering and Information Technology,Software Development,Cincinnati,OH,7.000000,80975.333333,129623.666667
4,Data Scientist(Remote),SynergisticIT,85.699248,Synergistic IT is a full-service staffing and ...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,Columbus,OH,150.000000,100977.222222,126081.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,Power BI Dashboard Analyst Developer,Tata Consultancy Services,85.699248,Technical/Functional Skills Power BI Dashboard...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",IT Services and IT Consulting,Columbus,OH,7.000000,95000.000000,120000.000000
695,Software Engineer Opportunities,Central Insurance,85.699248,"Location: Hybrid Work Model – Van Wert, Ohio o...",Entry level,Full-time,Engineering and Information Technology,Insurance,Van Wert,OH,7.000000,100977.222222,126081.444444
696,Software Quality Assurance (QA) Test Engineer,Medpace,85.699248,Voted a Top Cincinnati Workplace for 4 years r...,Not Applicable,Full-time,Quality Assurance,Pharmaceutical Manufacturing,Cincinnati,OH,32.267496,100977.222222,126081.444444
697,Software Engineer II,Honeywell,85.699248,"Join a team recognized for leadership, innovat...",Entry level,Full-time,Engineering and Information Technology,"Appliances, Electrical, and Electronics Manufa...",Mason,OH,14.000000,100977.222222,126081.444444


In [43]:
# check the # of null values
df.isna().sum()

,0
job_title,0
company_name,0
num_applicants,0
description,0
Seniority level,0
Employment type,0
Job function,0
Industries,0
city,0
state,0


In [44]:
#output the dataframe
df.to_csv("LinkedIn_cleaned_data.csv", index=False)